# Explorative Analysis of the Most Streamed Spotify Songs 2023 Dataset

In this notebook, we will perform an explorative analysis of the [Most Streamed Spotify Songs 2023 dataset](https://www.kaggle.com/datasets/nelgiriyewithana/top-spotify-songs-2023/data). This dataset contains information about the most streamed songs on Spotify in the year 2023. 

Our goal is to gain insights into the characteristics of these songs and identify any trends or patterns that may exist. We will start by loading and cleaning the data, and then proceed to perform various analyses and visualizations.
 

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px 
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('spotify-2023.csv', encoding='ISO-8859-1')
df.head()

In [ ]:
# check duplicates 
df.duplicated().sum()

In [ ]:
# check basic info about values in each column
columns_info = pd.DataFrame(df.dtypes, columns=['Data Type'])
columns_info['Null Values'] = df.isnull().sum()
columns_info['Unique Values'] = df.nunique()
columns_info['Count'] = df.count()
columns_info



In [ ]:
#check percentage count for each key value
df['key'].value_counts(normalize=True)

In [ ]:
#count of objects with more than 1 missing value
df[df.isnull().sum(axis=1) > 1].shape[0]

In [ ]:
df['in_shazam_charts'] = df['in_shazam_charts'].str.replace(',', '')
df['in_shazam_charts'] = pd.to_numeric(df['in_shazam_charts'])

columns_info = pd.DataFrame(df.dtypes, columns=['Data Type'])
columns_info['Null Values'] = df.isnull().sum()
columns_info['Unique Values'] = df.nunique()
columns_info['Count'] = df.count()
columns_info


### Frequencies of categorical attributes of the dataset (artist(s)_name, track_name, key, mode)	

In [ ]:
#freqencies for each categorical attribute if it has more than 10 unique values than top 10 freqs
cat_columns =  ['track_name', 'key', 'mode']
for col in cat_columns:
    freq_df = pd.DataFrame()
    freq_df['counts'] = df[col].value_counts()
    freq_df['percentage'] = df[col].value_counts(normalize=True).apply(lambda x: round(x*100,2))
    if df[col].nunique() > 20:
        print(freq_df.head(10))
    else:
        print(freq_df)

### Number of individual artists and respective frequencies

In [ ]:
df['artist(s)_name']
df_expanded_on_artists = df['artist(s)_name'].str.split(',', expand=True).stack().reset_index(level=1, drop=True)
df_expanded_on_artists = df_expanded_on_artists.str.strip()

freq_df = pd.DataFrame()
freq_df['counts'] = df_expanded_on_artists.value_counts()
freq_df['percentage'] = df_expanded_on_artists.value_counts(normalize=True).apply(lambda x: round(x*100,2))
freq_df.head(10)

In [ ]:
# explore numerical values by range, median, mean, std, median
df.describe().T

In [ ]:

df_charts_without_null = df[(df['in_shazam_charts'] > 0) & (df['in_deezer_charts'] > 0) & (df['in_spotify_charts'] > 0) & (df['in_apple_charts'] > 0)]
df_charts_without_null.describe().T

In [ ]:
df['streams'].str.isnumeric().all()

In [ ]:
df[~df['streams'].str.isnumeric()]

In [ ]:
# delete that row as 1 row is not significant
df = df.drop(df[~df['streams'].str.isnumeric()].index)

In [ ]:
# convert streams to numeric
df['streams'] = pd.to_numeric(df['streams'])
df['streams'].describe().apply('{0:,.2f}'.format)

In [ ]:
df['in_deezer_playlists'].str.isnumeric().all()


In [ ]:
df[~df['in_deezer_playlists'].str.isnumeric()]['in_deezer_playlists']


In [ ]:
df['in_deezer_playlists'] = df['in_deezer_playlists'].str.replace(',', '')
df['in_deezer_playlists'] = pd.to_numeric(df['in_deezer_playlists'])

In [ ]:
df.describe().T



In [ ]:
fig = df['released_year'].plot.hist(log=True, bins=20)
fig.set_xlabel('Year')
fig.set_title('Histogram of released_year frequency')

In [ ]:
# get numeric columns
corr = df.select_dtypes(include=['float64', 'int64']).corr()
plt.figure(figsize=(20,10))
sns.heatmap(corr, annot=True, cmap='coolwarm')

In [ ]:
corr

In [ ]:
df['discrete_streams'] = pd.cut(df['streams'], [0, 250e6, 500e6, 10e9], labels=['low', 'medium', 'high'])
df['discrete_streams'].value_counts(normalize=True)

In [ ]:
#numerical attributes
interesting_cols = [ 'streams', 'in_spotify_charts', 'in_spotify_playlists', 'released_year', 'bpm', 'energy_%', 'danceability_%', 'liveness_%', 'valence_%', 'acousticness_%', 'speechiness_%']
sns.pairplot(df, x_vars=interesting_cols, y_vars=interesting_cols,
            hue='discrete_streams')


In [ ]:
df['discrete_years'] = pd.cut(df['released_year'], [0, 2000,2010,2025], labels=['before 2000', '2000-2010', 'after 2010'])
df['discrete_years'].value_counts(normalize=True)



In [ ]:
px.violin(df, x='streams', y='discrete_years', color='discrete_years',box=True, orientation='h', title='Distribution of streams for year groups')


In [ ]:

px.scatter_3d(df, x='bpm', y='streams', z='energy_%', color='discrete_years')

### Exploring categorical attributes Key and Mode

In [ ]:
# key mapping [0,1,2,3,4,5,6,7,8,9,10,11],['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A','A#','B']
data_grouped_by_key = df.groupby('key')['streams'].sum().reset_index().sort_values('streams', ascending=False)
data_grouped_by_key.plot.bar(x='key', y='streams', colormap='Paired')

In [ ]:
# plot distribution of streams for individual modes
px.box(df, x='streams', y='mode', color='mode', orientation='h', title='Distribution of streams for individual modes')

In [ ]:
px.box(df, x='streams', y='key', color='key', orientation='h', title='Distribution of streams for individual keys')


In [ ]:
# test with paired t test with different variance
from scipy.stats import ttest_ind
from itertools import combinations
# test for each pair of keys once
# create pairs of keys without repetition
key_pairs = set(combinations(df['key'].unique(), 2))
for key_pair in key_pairs:
        i, j  = key_pair
        df_i = df[df['key'] == i]
        df_j = df[df['key'] == j]
        pvalue = ttest_ind(df_i['streams'], df_j['streams'], equal_var=False)[1]
        if pvalue > 0.05:
            print(f'The means of {i} - {j} are the same.(p-value {pvalue} > 0.05)')
        else:
            print(f'The means of {i} - {j} are different.(p-value {pvalue} < 0.05)')

In [ ]:
px.scatter_3d(df, x='streams', y='released_month', z='in_spotify_charts', color='key', title='Distribution of streams for individual keys and modes')


In [ ]:
# count tracks by keys in top 10 tracks by streams 
df.dropna(subset=['key','in_shazam_charts'], inplace=True)

keys = df['key'].unique()
bins = [0, 10, 30, 50, 100, len(df.index)]
sorted_by_streams = df.sort_values('streams', ascending=False)
topX_df = pd.DataFrame(columns=['topX', 'key', 'count','percentage'])
lastX_df = pd.DataFrame(columns=['lastX', 'key', 'count','percentage'])
for i, topX in enumerate(bins):
    if i + 1 >= len(bins):
        break
    topX_tracks = sorted_by_streams.head(bins[i+1])
    lastX_tracks = sorted_by_streams.tail(bins[i+1])
    for key in keys: 
        count = topX_tracks[topX_tracks['key'] == key].count()['key']
        percentage = topX_tracks[topX_tracks['key'] == key].count()['key'] / topX_tracks.count()['key']
        topX_df.loc[len(topX_df.index)] = [bins[i+1], key, count, percentage]
        count = lastX_tracks[lastX_tracks['key'] == key].count()['key']
        percentage = lastX_tracks[lastX_tracks['key'] == key].count()['key'] / lastX_tracks.count()['key']
        lastX_df.loc[len(lastX_df.index)] = [bins[i+1], key, count, percentage]
pd.concat([topX_df.head(11),lastX_df.head(11)])


In [ ]:
# plot percentage plot for each key
fig, axs = plt.subplots(2, figsize=(12,6))
topX_ticks = [f'Top {bin}' for bin in bins[1:]]
lastX_ticks = [f'Last {bin}' for bin in bins[1:]]
topX_axs = axs[0]
lastX_axs = axs[1]

for key in keys: 
    data = topX_df[topX_df['key'] == key]
    
    topX_axs.set_ylim(0, 0.6)
    topX_axs.scatter(topX_ticks, data['percentage'], label=key)
    topX_axs.plot(topX_ticks, data['percentage'], alpha=0.2)
    topX_axs.set_ylabel('percentage')
    topX_axs.set_xlabel('topX')
    topX_axs.legend(loc='lower right')

    data = lastX_df[lastX_df['key'] == key]
   
    lastX_axs.set_ylim(0, 0.5)
    lastX_axs.scatter(lastX_ticks, data['percentage'], label=key)
    lastX_axs.plot(lastX_ticks, data['percentage'], alpha=0.2)
    lastX_axs.set_ylabel('percentage')
    lastX_axs.set_xlabel('lastX')

plt.show()



In [ ]:
# cut top 100 songs by streams
sorted_by_streams = sorted_by_streams.reset_index()
sorted_by_streams['top100'] = pd.cut(sorted_by_streams.index, [0, 100, len(sorted_by_streams.index)], labels=['top100', 'rest'])



In [ ]:
px.scatter_3d(sorted_by_streams, x='danceability_%', y='acousticness_%', z='energy_%', color='discrete_years')


In [ ]:
px.histogram(sorted_by_streams,log_y=True, x='key', color='top100', title='Histogram of streams for individual keys and modes')

### Using Isolation forest to get outliers for numeric attributes

In [ ]:

from sklearn.ensemble import IsolationForest    
def train_and_predict_if(df, feature):
    rng = np.random.RandomState(42)
    clf = IsolationForest(max_samples=750, random_state=rng)
    clf.fit(df[[feature]])
    pred = clf.predict(df[[feature]])
    scores = clf.decision_function(df[[feature]])
    stats = pd.DataFrame()
    stats['val'] = df[feature]
    stats['score'] = scores
    stats['outlier'] = pred
    stats['min'] = df[feature].min()
    stats['max'] = df[feature].max()
    stats['mean'] = df[feature].mean()
    stats['feature'] = [feature] * len(df)
    return stats

def print_outliers(df, feature, n):
    print(feature)
    print(df[feature].head(n).to_string(), "\n")

num_columns = df.select_dtypes(include=['float64', 'int64'])
result = pd.DataFrame()
for feature in num_columns:
    stats = train_and_predict_if(df, feature)
    result = pd.concat([result, stats])

outliers = {team: grp.drop('feature', axis=1)
            for team, grp in result.sort_values(by='score').groupby('feature')}

n_outliers = 10
for feature in num_columns:
    print_outliers(outliers, feature, n_outliers)

There are no signaificant outliers for any of the numerical attributes. IsoForest score for each value < 0.36.

### Detecting outliers for numerical attributes

In [ ]:

def IQR_outlier_detection(df, feature):
    if df[feature].dtype != 'int64' and df[feature].dtype != 'float64':
        return pd.DataFrame()
    IQR = df[feature].quantile(0.75) - df[feature].quantile(0.25)
    lower_bound = df[feature].quantile(0.25) - (IQR * 1.5)
    upper_bound = df[feature].quantile(0.75) + (IQR * 1.5)
    return lower_bound, upper_bound, df[(df[feature] < lower_bound) | (df[feature] > upper_bound)]

IQR_outliers_stats = pd.DataFrame({'feature': [], 'count': []})
for attribute in num_columns:
    lower_bound, upper_bound, outliers = IQR_outlier_detection(df, attribute)
    IQR_outliers_stats = pd.concat([IQR_outliers_stats, pd.DataFrame({'feature': [attribute], 'count': [len(outliers)]})])
    
   
IQR_outliers_stats.reset_index(drop=True, inplace=True)
IQR_outliers_stats


In [ ]:
#z-score outlier detection
from scipy.stats import zscore

def z_score_outlier_detection(df, feature):
    if df[feature].dtype != 'int64' and df[feature].dtype != 'float64':
        return pd.DataFrame()
    z = np.abs(zscore(df[feature]))
    return df[(z > 3)]

z_score_outliers_stats = pd.DataFrame({'feature': [], 'count': []})
for attribute in num_columns:
    outliers = z_score_outlier_detection(df, attribute)
    z_score_outliers_stats = pd.concat([z_score_outliers_stats, pd.DataFrame({'feature': [attribute], 'count': [len(outliers)]})])

z_score_outliers_stats.reset_index(drop=True, inplace=True)
z_score_outliers_stats


In [ ]:
# combined outlier detection feature z-score IQR from dataframes
combined_outliers_stats = pd.merge(IQR_outliers_stats, z_score_outliers_stats, on='feature', how='outer', )
combined_outliers_stats.rename(columns={'count_x': 'IQR', 'count_y': 'z-score'}, inplace=True)
combined_outliers_stats